In [1]:
'''
Created on Jul 27, 2018

@author: omert
'''
import json
import random
import argparse
from operator import itemgetter
from gensim.models import LdaModel
from gensim import corpora
from datetime import datetime

TEXT = True
#DIR = "/home/omertl123/yelp/dataset/"
DIR = "../data/"
AMAZON_TYPE="Home_and_Kitchen"
OUTPUT_DIR = "amazon_dataset/" + AMAZON_TYPE + "/"
FILE_NAME = DIR+"reviews_" + AMAZON_TYPE + "_5.json"#"Electronics_5.json"
reviews_count = 0
users = {}
pois = {}
reviews_file = open(FILE_NAME,encoding="utf8")
users_count = 0
data_loaded = 0
poi_count = 0
temp_data = []
for line in reviews_file:
    data_loaded+=1
    if (data_loaded % 100000 == 0):
        print("Processing record {}".format(data_loaded))
        print("Loaded {} users, {} locations and {} reviews".format(users_count,poi_count,reviews_count))
    record = json.loads(line)
    user_id = record["reviewerID"]
    poi_id = record["asin"]
    ts = record["unixReviewTime"]
    score = float(record["overall"])
    date = str(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d'))
    text = record["reviewText"].replace("\t","").replace("\n","")
    if score>=0:
        if user_id not in users:
            users[user_id]=0
            users_count+=1
        if poi_id not in pois:
            pois[poi_id]=0
            poi_count+=1
        if user_id in users and poi_id in pois:
            if TEXT:
                temp_data.append((user_id,poi_id,date,text))
            else:
                temp_data.append((user_id,poi_id,date,""))
            users[user_id]+=1
            pois[poi_id]+=1
            reviews_count+=1

reviews_file.close()

print("Loaded {} reviews".format(reviews_count))

Processing record 100000
Loaded 44898 users, 3954 locations and 99999 reviews


KeyboardInterrupt: 

In [25]:
POI_LIMIT = 0
MIN_REVIEWS_POI = 10
USER_LIMIT = 0
MIN_REVIEWS_USER = 15
TOP_TOPICS = 25
MIN_USERS_FOR_SET = 15
NEG_RECORDS = 3

In [26]:
if POI_LIMIT>0:
    pois_sorted = sorted(pois.items(), key=itemgetter(1),reverse=True)
    counter=0
    for key,value in pois_sorted:
        counter+=1
        if (counter>POI_LIMIT):
            pois[key] = 0

if USER_LIMIT>0:
    users_sorted = sorted(users.items(), key=itemgetter(1),reverse=True)
    counter=0
    for key,value in users_sorted:
        counter+=1
        if (counter>USER_LIMIT):
            users[key] = 0
        
filtered_data = {}
filtered_records = 0
user_indexes = {}
user_index_rev = {}
poi_indexes = {}
poi_index_rev = {}
user_counter = 0
poi_counter = 0
user_distinct = {}
for record in temp_data:
    user_id = record[0]
    poi_id = record[1]
    date = record[2]
    text = record[3]
    if (users[user_id] >= MIN_REVIEWS_USER and pois[poi_id]>=MIN_REVIEWS_POI):
        if user_id not in user_indexes:
            user_indexes[user_id] = user_counter
            user_index_rev[user_counter] = user_id
            user_counter += 1
        if poi_id not in poi_indexes:
            poi_indexes[poi_id] = poi_counter
            poi_index_rev[poi_counter] = poi_id
            poi_counter += 1
        user_idx = user_indexes[user_id]
        poi_idx = poi_indexes[poi_id]
        if user_idx not in filtered_data:
            filtered_data[user_idx] = []
            user_distinct[user_idx] = set()
        if poi_idx not in user_distinct[user_idx]:
            value = (poi_idx,date,text)
            filtered_data[user_idx].append(value)
            user_distinct[user_idx].add(poi_idx)
        filtered_records+=1

print("loaded {} output records with {} users and {} locations".format(filtered_records,user_counter,poi_counter))
print("sparsity:{}%".format((poi_counter/(user_counter*poi_counter))*100))

loaded 879878 output records with 21756 users and 30761 locations
sparsity:0.00459643316786174%


In [27]:
def write_record_to_file(user,data,idx,K,file,remove_text,last_k_str=""):
    if last_k_str=="":
        last_k = []
        for i in range(idx-K,idx):
            last_k.append(data[i][0])
        last_k_str=','.join(str(j) for j in last_k)
    if remove_text:
        file.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(user,data[idx][0],1,data[idx][1],last_k_str,""))
    else:
        file.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(user,data[idx][0],1,data[idx][1],last_k_str,data[idx][2].encode("utf-8")))
    return last_k_str

def write_negative_samples(user,data,file,samples,poi_count,last_k):
    counter = 0
    for j in range(samples):
        randomize = True
        while randomize:
            r = random.randint(1,poi_count)
            for k in range(len(data)):
                if data[k][0] == r:
                    break
            if k==len(data)-1 and data[k][0]!=r:
                randomize = False
        file.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(user,r,0,"2018-08-01",last_k,"".encode("utf-8")))
        counter += 1
    return counter

In [30]:
import pickle

REMOVE_TEXT = True

In [31]:
for K in [4,6,8,10,12]:
    OUTPUT_PREFIX = OUTPUT_DIR+str(filtered_records//1000)+"_amazon_"
    FILES_PREFIX = OUTPUT_PREFIX + str(K) + "_"
    train_file = open(FILES_PREFIX + "train.txt","w+")
    test_file = open(FILES_PREFIX + "test.txt","w+")
    valid_file = open(FILES_PREFIX + "valid.txt","w+")
    training_records = 0
    test_records = 0
    users_wrote = 0
    valid_records = 0
    max_poi_index = len(poi_indexes) - 1
    user_index2 = {}
    user_index_rev2 = {}
    for user_raw in filtered_data:
        data = filtered_data[user_raw]
        data.sort(key=itemgetter(1))
        if len(data)>=MIN_USERS_FOR_SET:
            user = users_wrote
            user_index2[user_raw] = user
            user_index_rev2[users_wrote] = user_index_rev[user_raw]
            users_wrote+=1
            for idx in range(K,len(data)-2):
                last_k = write_record_to_file(user,data,idx,K,train_file,REMOVE_TEXT)
                training_records += 1
                #negative sampling
                training_records += write_negative_samples(user,data,train_file,NEG_RECORDS,max_poi_index,last_k)
            last_k = write_record_to_file(user,data,len(data)-2,K,valid_file,REMOVE_TEXT)
            valid_records+=1
            valid_records+=write_negative_samples(user,data,valid_file,NEG_RECORDS,max_poi_index,last_k)
            last_k = write_record_to_file(user,data,len(data)-1,K,test_file,REMOVE_TEXT,last_k)
            test_records+=1
            test_records+=write_negative_samples(user,data,test_file,NEG_RECORDS,max_poi_index,last_k)
    train_file.close()
    valid_file.close()
    test_file.close()

    print("Wrote {} training, {} validation and {} test records".format(training_records,valid_records,test_records))
    print("{} users, {} locations".format(users_wrote,len(poi_indexes)))
    print("Wrote to {}".format(str(filtered_records//1000)))

Wrote 2469196 training, 78264 validation and 78264 test records
19566 users, 30761 locations
Wrote to 879


In [7]:
import ast

poi_to_name = {}
with open(DIR+"meta_Home_and_Kitchen.json",'r',encoding="utf8") as f:
    for line in f:
        record = ast.literal_eval(line)#json.loads(line)
        poi_id = record['asin']
        if('title' in record):
            poi_name = record['title']
        else:
            poi_name = "Unknown"
        if poi_id in poi_indexes:
            poi_to_name[poi_id] = poi_name

In [8]:
with open(OUTPUT_PREFIX + "poi_index_rev.pkl","wb+") as f:
    pickle.dump(poi_index_rev, f)
with open(OUTPUT_PREFIX + "user_index_rev.pkl","wb+") as f:
    pickle.dump(user_index_rev2, f)
with open(OUTPUT_PREFIX + "poi_names.pkl","wb+") as f:
    pickle.dump(poi_to_name, f)

In [10]:
import numpy as np
topics_lst = [10,25,30,50,70,90]
for LDA_TOPICS in topics_lst:
    LDA_PATH = "data_cnn/lda/amazon_home/" # + "models_" + str(LDA_TOPICS) + "/"

    dictionary_path = LDA_PATH + "dictionary_" + str(LDA_TOPICS) + ".dict"
    corpus_path = LDA_PATH + "corpus" + str(LDA_TOPICS) + ".lda-c"
    dictionary = corpora.Dictionary.load(dictionary_path)
    corpus = corpora.BleiCorpus(corpus_path)
    lda_model_path = LDA_PATH + "lda_model_" + str(LDA_TOPICS) + "_topics.lda"
    lda = LdaModel.load(lda_model_path)
    poi_topics = {}
    for user_data in filtered_data.values():
        for record in user_data:
            poi_id = record[0]
            if poi_id not in poi_topics:
                poi_topics[poi_id] = np.zeros(LDA_TOPICS)
            review = record[2]
            lda_topics = lda[dictionary.doc2bow(review.split(" "))]
            for i in range(LDA_TOPICS):
                poi_topics[poi_id][i] += lda_topics[i][1]
    with open(OUTPUT_PREFIX + "topics_"+str(LDA_TOPICS)+".pkl","wb+") as f:
        pickle.dump(poi_topics, f)